In [1]:
%cd semantic_code_search/code_search/modeling

/teamspace/studios/this_studio/semantic_code_search/code_search/modeling


In [15]:
from omegaconf import OmegaConf
from enum import Enum, auto
from abc import ABC, abstractmethod
from typing import Type
import torch
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
from transformers import AutoModel, AutoTokenizer, PreTrainedTokenizerBase,PreTrainedTokenizer
from transformers.data import DataCollatorForSeq2Seq
from peft import (PeftModel, PeftConfig, PrefixTuningConfig, 
                 LoraConfig, get_peft_model, TaskType)
from tokenizers import Tokenizer
from omegaconf import DictConfig
from omegaconf import OmegaConf
from typing import Optional, Dict, Type
from optimizers import *
from model_manager import *
from losses import *
import pandas as pd

In [16]:
tokenizer = AutoTokenizer.from_pretrained("sentence-transformers/all-MiniLM-L6-v2")

loading file vocab.txt from cache at /home/zeus/.cache/huggingface/hub/models--sentence-transformers--all-MiniLM-L6-v2/snapshots/fa97f6e7cb1a59073dff9e6b13e2715cf7475ac9/vocab.txt
loading file tokenizer.json from cache at /home/zeus/.cache/huggingface/hub/models--sentence-transformers--all-MiniLM-L6-v2/snapshots/fa97f6e7cb1a59073dff9e6b13e2715cf7475ac9/tokenizer.json
loading file added_tokens.json from cache at None
loading file special_tokens_map.json from cache at /home/zeus/.cache/huggingface/hub/models--sentence-transformers--all-MiniLM-L6-v2/snapshots/fa97f6e7cb1a59073dff9e6b13e2715cf7475ac9/special_tokens_map.json
loading file tokenizer_config.json from cache at /home/zeus/.cache/huggingface/hub/models--sentence-transformers--all-MiniLM-L6-v2/snapshots/fa97f6e7cb1a59073dff9e6b13e2715cf7475ac9/tokenizer_config.json
loading file chat_template.jinja from cache at None


In [29]:
tokenizer(["hello world","hello"], return_tensors="pt",padding="longest")

{'input_ids': tensor([[ 101, 7592, 2088,  102],
        [ 101, 7592,  102,    0]]), 'token_type_ids': tensor([[0, 0, 0, 0],
        [0, 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1, 1],
        [1, 1, 1, 0]])}

In [17]:

class CodeSearchDataset(Dataset):
    def __init__(self,
                 config: DictConfig,):
        """
        Initializes the dataset.

        Args:
            file_paths: A list of paths to JSONL files.
            tokenizer: A CodeSearchTokenizer instance.
        """
        self.config = config
        self.data = self._load_data_from_jsonl(self.config.file_paths)

    def _load_data_from_jsonl(self, file_paths: List[str]) -> pd.DataFrame:
        """Loads data from one or more JSONL files into a pandas DataFrame."""
        all_data = []
        for file_path in file_paths:
            try:
                data = pd.read_json(file_path, lines=True)
                all_data.append(data)
            except ValueError:
                print(f"Warning: File wans't read: {file_path}")
                continue
        return pd.concat(all_data, ignore_index=True)

    def __len__(self):
        """Returns the length of the dataset."""
        return len(self.data)

    def __getitem__(self, idx):
        """
        Retrieves an item from the dataset.

        Args:
            idx: The index of the item to retrieve.

        Returns:
            A dictionary containing the tokenized inputs.
        """
        item = self.data.iloc[idx]
        query = item['query']
        code = item['code']
        return {
            "query": query,
            "code": code
        }


In [18]:
config = OmegaConf.load("/teamspace/studios/this_studio/semantic_code_search/conf/modeling.yaml")

In [21]:
dataset = CodeSearchDataset(config=config.data.train_dataset)

In [23]:
dataloader = DataLoader(dataset, batch_size=8, shuffle=False)

In [24]:
o = next(iter(dataloader))

In [25]:
o

{'query': ['get the index of an item in a list.',
  'performs linear regression using glm.',
  'returns strings that match a regular expression.',
  'changes the color of a string.',
  'plots linear regression on an existing plot.',
  'create a yes or no response.',
  'checks if an XML element has a beginning property.',
  'concatenate columns in a SQL query.'],
 'code': ['def get_index(lst, item):\n    return lst.index(item)\n```',
  'def glm_linear_regression(X, y):\n    import statsmodels.api as sm\n    X = sm.add_constant(X)\n    model = sm.OLS(y, X).fit()\n    return model\n```',
  'def filter_strings(lst, pattern):\n    import re\n    return [s for s in lst if re.match(pattern, s)]\n```',
  'def change_string_color(s, color):\n    return f"\\033[{color}m{s}\\033"\n```',
  "def plot_linear_regression(ax, x, y):\n    from numpy import polyfit, poly1d\n    coefficients = polyfit(x, y, 1)\n    linear_func = poly1d(coefficients)\n    ax.plot(x, linear_func(x), color='red')\n```",
  'd

In [26]:
collator = DataCollatorForSeq2Seq(tokenizer)

In [27]:
collator(o)

KeyError: 0

In [13]:
d={
    "q" : ["this is the first sentence"],
    "k" : ["this is key"]
}

In [14]:
collator(d)

KeyError: 0